## Libraries

In [ ]:
import holoviews as hv
from holoviews import dim
hv.extension('bokeh')
from bokeh.plotting import show
from bokeh.io import export_svgs
import pyperclip
import pandas as pd
from collections import Counter
import itertools

## Importig data from excel clipboard

In [ ]:
df = pd.read_clipboard()
df

## GET LABELS AND COLORS

In [ ]:
tracker_attrs = df.loc[df['FactorType'] == "Tracking"]
attrs_path = df.loc[df['FactorType'] == "Path"]

tracker_colors = dict(zip(tracker_attrs["FactorName"], tracker_attrs["HexColor"]))
attr_colors = dict(zip(tracker_attrs["Attribute"], itertools.repeat("#808080")))
path_colors = dict(zip(attrs_path["FactorName"], attrs_path["HexColor"]))

In [ ]:
all_labels = []
all_labels  = list(tracker_colors.keys()) +  list(attr_colors.keys()) + list(path_colors.keys())
all_labels

In [ ]:
colormap = []
colormap = list(tracker_colors.values()) +  list(attr_colors.values()) + list(path_colors.values())
print(colormap)

## GET SOURCE AND TARGET

In [ ]:
mapping_tracker = {item:i for i, item in enumerate(tracker_attrs["FactorName"].unique())}
sources = []
targets = []
for item in tracker_attrs["FactorName"]:
    sources.append(mapping_tracker[item])

max_sources = max(sources)

mapping_attrs = {item:i for i, item in enumerate(tracker_attrs["Attribute"].unique())}

for item in tracker_attrs["Attribute"]:
    sources.append(mapping_attrs[item] + (max_sources+1))
    targets.append(mapping_attrs[item] + (max_sources+1))

max_targets = max(targets)

mapping_path = {item:i for i, item in enumerate(attrs_path["FactorName"].unique())}
for item in attrs_path["FactorName"]:
    targets.append(mapping_path[item] + (max_targets+1))


source_target = list(zip(sources, targets))
print(source_target)

In [ ]:
print(f"{len(sources)}, {len(targets)}")

In [ ]:
source_target_weight = [(t[0], t[1], 1) for t in source_target]
print(source_target_weight)

## PLOT SANKEY

In [ ]:
hv.extension("bokeh", "matplotlib")

nodes = all_labels
nodes = hv.Dataset(enumerate(nodes), 'index', 'label')
edges = source_target_weight

value_dim = hv.Dimension('origin')
sankey = hv.Sankey((edges, nodes), ['From', 'To'], vdims=value_dim).opts(
    fontsize={'title': 16, 'labels': 18}, 
    labels='label', 
    label_position='outer', 
    width=2700, 
    height=1080, 
    cmap=colormap,
    node_color=dim('From').str(), 
    title="Tracker (Categoría) vs BSA (Marca)", 
    node_width = 15, 
    node_alpha = 1.0,
    show_values = False,
    node_padding = 9,
    label_text_font_size = '22pt')

show(hv.render(sankey))

In [ ]:
renderer = hv.renderer("bokeh")
renderer.save(sankey.options(toolbar=None), "sankey", fmt="png")